In [1]:
import io
import os
import sys
import numpy as np
import tensorflow as tf
from PIL import Image
from flask import Flask, jsonify, request, render_template, Response, flash, redirect, url_for, session
import base64
import cv2
from datetime import datetime
import time
from flask_mysqldb import MySQL
import MySQLdb.cursors
from flask_login import LoginManager, login_user, login_required, logout_user, current_user
from pyfcm import FCMNotification
import threading

In [2]:
app = Flask(__name__, template_folder="./Website")
app.secret_key = "jun"

# initialize login_manager
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

app.config["MYSQL_HOST"] = "localhost"
app.config["MYSQL_USER"] = "root"
app.config["MYSQL_PASSWORD"] = ""
app.config["MYSQL_DB"] = "tics"
mysql = MySQL(app)

# initialize the FCM client with your server key
push_service = FCMNotification(api_key= "AAAAbrgCjxQ:APA91bFvuOioyd30-Uel3ePE2KRHXW8L7AeT5f1okljjD_veyshDuzE8XcHQdIiDrXXHruQjMz6nf4dlgcuLlE_jfrnfmFG-3eJr9icf_aAvw2WVjOBCPCaSf_5SYHnR4PGuWRCPjzTv")

class User:
    def __init__(self, StudentID, StudentName, ClassID, StudentParentName, StudentParentNo, active=True):
        self.StudentID = StudentID
        self.StudentName = StudentName
        self.ClassID = ClassID
        self.StudentParentName = StudentParentName
        self.StudentParentNo = StudentParentNo
        self.active = active
        self.is_authenticated = True
        
    def is_active(self):
        return self.active

    def get_id(self):
        return self.StudentID
    
    def getStudentName(self):
        return self.StudentName

@app.route("/")
def home():
    return render_template("Home.html", user=current_user)

@app.route("/register", methods=["GET", "POST"])
def register():
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute("SELECT ClassID, ClassName FROM class")
    classes = cursor.fetchall()
    if request.method == "POST" and "StudentName" in request.form:
        StudentName = request.form.get("StudentName")
        StudentParentName = request.form.get("StudentParentName")
        StudentParentNo = request.form.get("StudentParentNo")
        ClassID = request.form.get("ClassID")
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute("SELECT * FROM students WHERE StudentName=%s", (StudentName,))
        user = cursor.fetchone()
        if user:
            flash("User already exists. Please log in!", "danger")
            return redirect(url_for("login"))
        else:
            cursor.execute("INSERT INTO students(StudentName, StudentParentName, StudentParentNo, ClassID) VALUES(%s, %s, %s, %s)", (StudentName, StudentParentName, StudentParentNo, ClassID,))
            mysql.connection.commit()
            flash("User registered successfully. Please log in!", "success")
            return redirect(url_for("login"))
    return render_template("Register.html", user=current_user, classes=classes)

@app.route("/login", methods=["GET", "POST"])
def login():
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute("SELECT ClassID, ClassName FROM class")
    classes = cursor.fetchall()
    if request.method == "POST" and "StudentName" in request.form:
        StudentName = request.form.get("StudentName")
        ClassID = request.form.get("ClassID")
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute("SELECT * FROM students WHERE StudentName=%s AND ClassID=%s", (StudentName, ClassID,))
        user = cursor.fetchone()
        if user:
            user = User(user["StudentID"], user["StudentName"], user["ClassID"], user["StudentParentName"], user["StudentParentNo"])
            session["StudentID"] = int(user.get_id())
            login_user(user, remember = True)  
            flash("Logged in successfully!", "success")
            return redirect(url_for("userhome"))
        else:
            flash("Login Unsuccessful. Please check your credentials!", "danger")
    return render_template("login.html", user=current_user, classes=classes)

@app.route("/logout")
@login_required
def logout():
    logout_user()
    flash("You have logged out successfully!", "success")
    return redirect(url_for("home"))

@login_manager.user_loader
def load_user(userid):
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute("SELECT * FROM students WHERE StudentID=%s", (userid,))
    student = cursor.fetchone()
    if student:
        return User(student["StudentID"], student["StudentName"], student["ClassID"], student["StudentParentName"], student["StudentParentNo"])
    return None
    
@app.route("/userhome")
@login_required
def userhome():
    return render_template("User.html", user=current_user)

@app.route("/update", methods=["GET", "POST"])
@login_required
def update():
    if request.method == "POST":
        StudentName = request.form.get("StudentName")
        StudentParentName = request.form.get("StudentParentName")
        StudentParentNo = request.form.get("StudentParentNo")
        ClassName = request.form.get("Class")
        StudentID = current_user.get_id()

        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute("SELECT ClassID FROM class WHERE ClassName=%s", (ClassName,))
        result = cursor.fetchone()
        
        if result:
            ClassID = result["ClassID"]
            cursor.execute("UPDATE students SET StudentName=%s, StudentParentName=%s, StudentParentNo=%s, ClassID=%s WHERE StudentID=%s", (StudentName, StudentParentName, StudentParentNo, ClassID, StudentID))
            mysql.connection.commit()
            
            flash("Profile updated successfully!", "success")
            return redirect(url_for("userhome"))
        else:
            flash("Invalid class name!", "error")
            
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute("SELECT * FROM students WHERE StudentID=%s", (current_user.get_id(),))
    user = cursor.fetchone()

    cursor.execute("SELECT ClassName FROM class")
    classes = cursor.fetchall()
    ClassNames = [row["ClassName"] for row in classes]

    return render_template("Update.html", user=current_user, ClassNames=ClassNames)

def detectgame(video_capture, userid, username):
    labels_dict = {0: "No Tic", 1: "Tic"}
    color_dict = {0: (0, 0, 255), 1: (0, 255, 0)}
    img_size = 80
    face_cascade = cv2.CascadeClassifier(r"./haarcascade_frontalface_default.xml")
    model = tf.keras.models.load_model("./model/TIC.h5")
    tic_detected = False
    tic_start_time = None
    GameTicCount= 0

    while True:
        success, frame = video_capture.read()
        if not success:
            break
        else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

            for (x, y, w, h) in faces:
                roi = frame[y:y + h, x:x + w]
                roi = cv2.resize(roi, (img_size, img_size))
                roi = roi / 255.0
                roi = roi.reshape(1, img_size, img_size, 3)
                tic_prob = model.predict(roi)[0][1]
                tic_label = labels_dict[int(tic_prob > 0.5)]
                tic_color = color_dict[int(tic_prob > 0.5)]

                if tic_label == "Tic":
                    if not tic_detected:
                        tic_start_time = time.time()
                        tic_detected = True
                                
                    else:
                        if time.time() - tic_start_time > 2:
                            TicDate = datetime.now().date()
                            TicTime = datetime.now().time()
                            img_np = np.array(frame)
                            success, img_encoded = cv2.imencode(".jpg", img_np)
                            if success:
                                GameTicCount+=1
                                img_data = img_encoded.tobytes()
                                cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
                                cursor.execute("INSERT INTO tic(TicDate, TicTime, Picture, StudentID) VALUES (%s, %s, %s, %s)", (TicDate, TicTime, img_data, userid))
                                mysql.connection.commit()
                                # Send FCM notification to device with registration ID
                                registration_id = "dE3O29gXSSSPlh9SVhYarL:APA91bFsUvxgsyXbep_p_HHjGK94lXlM4KT3cuqLouxPHTDAp87LDzHUzts4Xni11fqu9tYo0BcbaYuC8BB8QcTlcrZxIVEXgtvV3VAihaB9dXIil10o3iR7OL2Z-ah6cRmxg6S54z0f"
                                message_title = "TICS"
                                message_body = f"Tic detected for {username} playing Game"
                                result = push_service.notify_single_device(registration_id=registration_id, message_title=message_title, message_body=message_body)
                                print(result)
                                tic_detected = False
                else:
                    tic_detected = False
                cv2.rectangle(frame, (x, y), (x + w, y + h), tic_color, 2)
                cv2.putText(frame, tic_label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, tic_color, 2)

            cv2.imshow("Camera", cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
            if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
                
    cursor = mysql.connection.cursor()
    cursor.execute("SELECT GameID FROM game ORDER BY GameID DESC LIMIT 1")
    result = cursor.fetchone()
    cursor.execute("UPDATE game SET GameTicCount=%s, GameEndTime=%s WHERE GameID=%s", (GameTicCount, datetime.now().time(), result))
    mysql.connection.commit()
    cv2.destroyAllWindows()
    video_capture.release()

@app.route("/game", methods=['GET', 'POST'])
@login_required
def game():
    GameDate = datetime.now().date()
    if request.method == 'POST':
        if 'start_game' in request.form:
            GameStartTime = datetime.now().time()
            cursor = mysql.connection.cursor()
            cursor.execute("INSERT INTO game (GameDate, GameStartTime, StudentID) VALUES (%s, %s, %s)", (GameDate, GameStartTime, current_user.get_id()))
            mysql.connection.commit()
            video_capture = cv2.VideoCapture(0)
            detectgame(video_capture, current_user.get_id(), current_user.getStudentName())
    return render_template("Game.html", user=current_user)

@app.route("/about")
def about():
    return render_template("About.html", user=current_user)

@app.route("/tichistory")
@login_required
def tichistory():
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute("SELECT * FROM tic WHERE StudentID=%s ORDER BY TicDate DESC, TicTime DESC", (current_user.get_id(),))
    history = cursor.fetchall()
    for row in history:
        img_data = base64.b64encode(row["Picture"]).decode("utf-8")
        row["Picture"] = img_data
    return render_template("TicHistory.html", user=current_user, history=history)

@app.route("/drawinghistory")
@login_required
def drawinghistory():
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute("SELECT Drawing, DrawingDate, DrawingTime FROM drawings WHERE StudentID=%s ORDER BY DrawingDate DESC, DrawingTime DESC", (current_user.get_id(),))
    history = cursor.fetchall()
    for row in history:
        img_data = base64.b64encode(row["Drawing"]).decode("utf-8")
        row["Drawing"] = img_data
    return render_template("DrawingHistory.html", user=current_user, history=history)

@app.route("/draw", methods=["GET", "POST"])
@login_required
def draw():
    if request.method == "POST":
        canvas_data = request.form["jsvar"]
        drawing_bytes = base64.b64decode(canvas_data.split(",")[1])
        DrawingDate = datetime.now().date() 
        DrawingTime = datetime.now().time()        
        cursor = mysql.connection.cursor()
        cursor.execute(
            "INSERT INTO drawings (Drawing, DrawingDate, DrawingTime, StudentID) VALUES (%s, %s, %s, %s)",
            (drawing_bytes, DrawingDate, DrawingTime, current_user.get_id())
        )
        mysql.connection.commit()
        flash("Drawing saved successfully!", "success")
        return redirect(url_for("draw"))
    else:
        return render_template("Draw.html", user=current_user)
    
def detectcamera(video_capture, userid, username):
    labels_dict = {0: "No Tic", 1: "Tic"}
    color_dict = {0: (0, 0, 255), 1: (0, 255, 0)}
    img_size = 80
    face_cascade = cv2.CascadeClassifier(r"./haarcascade_frontalface_default.xml")
    model = tf.keras.models.load_model("./model/TIC.h5")
    tic_detected = False
    tic_start_time = None

    while True:
        success, frame = video_capture.read()
        if not success:
            break
        else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

            for (x, y, w, h) in faces:
                roi = frame[y:y + h, x:x + w]
                roi = cv2.resize(roi, (img_size, img_size))
                roi = roi / 255.0
                roi = roi.reshape(1, img_size, img_size, 3)
                tic_prob = model.predict(roi)[0][1]
                tic_label = labels_dict[int(tic_prob > 0.5)]
                tic_color = color_dict[int(tic_prob > 0.5)]

                if tic_label == "Tic":
                    if not tic_detected:
                        tic_start_time = time.time()
                        tic_detected = True
                                
                    else:
                        if time.time() - tic_start_time > 2:
                            TicDate = datetime.now().date()
                            TicTime = datetime.now().time()
                            img_np = np.array(frame)
                            success, img_encoded = cv2.imencode(".jpg", img_np)
                            if success:
                                img_data = img_encoded.tobytes()
                                cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
                                cursor.execute("INSERT INTO tic(TicDate, TicTime, Picture, StudentID) VALUES (%s, %s, %s, %s)", (TicDate, TicTime, img_data, userid))
                                mysql.connection.commit()
                                # Send FCM notification to device with registration ID
                                registration_id = "dE3O29gXSSSPlh9SVhYarL:APA91bFsUvxgsyXbep_p_HHjGK94lXlM4KT3cuqLouxPHTDAp87LDzHUzts4Xni11fqu9tYo0BcbaYuC8BB8QcTlcrZxIVEXgtvV3VAihaB9dXIil10o3iR7OL2Z-ah6cRmxg6S54z0f"
                                message_title = "TICS"
                                message_body = f"Tic detected for {username}"
                                result = push_service.notify_single_device(registration_id=registration_id, message_title=message_title, message_body=message_body)
                                print(result)
                                tic_detected = False
                else:
                    tic_detected = False
                cv2.rectangle(frame, (x, y), (x + w, y + h), tic_color, 2)
                cv2.putText(frame, tic_label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, tic_color, 2)

            cv2.imshow("Camera", cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
            if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
                    
    cv2.destroyAllWindows()
    video_capture.release()
    
@app.route("/camera", methods=["GET", "POST"])
@login_required
def camera():
    if request.method == "POST":
        if request.form.get("submit_button") == "Start":
            flash("Live Detection is sucessful!", "success")
            # Start capturing video from default camera
            video_capture = cv2.VideoCapture(0)
            # Call detect_tic function to start detecting tics
            detectcamera(video_capture, current_user.get_id(), current_user.getStudentName())
            return render_template("Camera.html", user=current_user)
    else:
        return render_template("Camera.html", user=current_user)

if __name__ == "__main__":
    from werkzeug.serving import run_simple
    app.debug= True    
    run_simple("localhost", 5001, app)

 * Running on http://localhost:5001
Press CTRL+C to quit
127.0.0.1 - - [03/Mar/2023 22:18:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:18:00] "GET /static/index.css HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:18:01] "GET /static/Index.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:18:05] "GET /camera HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:18:05] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:18:06] "GET /static/Index.js HTTP/1.1" 304 -


{'multicast_ids': [5876260234754570642], 'success': 1, 'failure': 0, 'canonical_ids': 0, 'results': [{'message_id': '0:1677853117767096%3083aef63083aef6'}], 'topic_message_id': None}
{'multicast_ids': [5555092665375368216], 'success': 1, 'failure': 0, 'canonical_ids': 0, 'results': [{'message_id': '0:1677853120112082%3083aef63083aef6'}], 'topic_message_id': None}


127.0.0.1 - - [03/Mar/2023 22:18:34] "POST /camera HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:18:34] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:18:34] "GET /static/Index.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:20:02] "GET /game HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:20:02] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:20:02] "GET /static/Index.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:20:03] "GET /static/Build/WebGL.loader.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:20:03] "GET /static/Build/WebGL.framework.js.unityweb HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:20:03] "GET /static/Build/WebGL.wasm.unityweb HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:20:03] "GET /static/Build/WebGL.data.unityweb HTTP/1.1" 304 -


{'multicast_ids': [3938690865890851608], 'success': 1, 'failure': 0, 'canonical_ids': 0, 'results': [{'message_id': '0:1677853292945663%3083aef63083aef6'}], 'topic_message_id': None}
{'multicast_ids': [3641301446304549978], 'success': 1, 'failure': 0, 'canonical_ids': 0, 'results': [{'message_id': '0:1677853295396463%3083aef63083aef6'}], 'topic_message_id': None}


127.0.0.1 - - [03/Mar/2023 22:21:31] "POST /game HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:21:32] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:21:32] "GET /static/Index.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:21:32] "GET /static/Build/WebGL.loader.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:21:32] "GET /static/Build/WebGL.framework.js.unityweb HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:21:32] "GET /static/Build/WebGL.wasm.unityweb HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:21:32] "GET /static/Build/WebGL.data.unityweb HTTP/1.1" 304 -


...

127.0.0.1 - - [03/Mar/2023 22:25:50] "POST /game HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:25:51] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:25:51] "GET /static/Index.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:25:51] "GET /static/Build/WebGL.loader.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:25:51] "GET /static/Build/WebGL.framework.js.unityweb HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:25:51] "GET /static/Build/WebGL.wasm.unityweb HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:25:51] "GET /static/Build/WebGL.data.unityweb HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:25:52] "GET /camera HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:25:52] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:25:52] "GET /static/Index.js HTTP/1.1" 304 -


{'multicast_ids': [3270181763042829466], 'success': 1, 'failure': 0, 'canonical_ids': 0, 'results': [{'message_id': '0:1677853580337314%3083aef63083aef6'}], 'topic_message_id': None}


127.0.0.1 - - [03/Mar/2023 22:26:18] "POST /camera HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:26:19] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:26:19] "GET /static/Index.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:34:46] "GET /game HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:34:46] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:34:46] "GET /static/Index.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:34:46] "GET /static/Build/WebGL.loader.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:34:47] "GET /static/Build/WebGL.wasm.unityweb HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:34:47] "GET /static/Build/WebGL.framework.js.unityweb HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:34:47] "GET /static/Build/WebGL.data.unityweb HTTP/1.1" 304 -


{'multicast_ids': [1694824260240145129], 'success': 1, 'failure': 0, 'canonical_ids': 0, 'results': [{'message_id': '0:1677854113677250%3083aef63083aef6'}], 'topic_message_id': None}
{'multicast_ids': [1417050477647817371], 'success': 1, 'failure': 0, 'canonical_ids': 0, 'results': [{'message_id': '0:1677854116054527%3083aef63083aef6'}], 'topic_message_id': None}


127.0.0.1 - - [03/Mar/2023 22:35:15] "POST /game HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:35:15] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:35:15] "GET /static/Index.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:35:15] "GET /static/Build/WebGL.loader.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:35:15] "GET /static/Build/WebGL.framework.js.unityweb HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:35:15] "GET /static/Build/WebGL.wasm.unityweb HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:35:15] "GET /static/Build/WebGL.data.unityweb HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:42:17] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:47:32] "GET /game HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:47:32] "GET /static/index.css HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2023 22:47:33] "GET /static/Index.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:47:33] "GET /static/Build/WebGL.loader.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Mar/2023 22:47: